# 감성분석

In [1]:
# 1.단어사전기반 분석 - 감성사전을 이용하여 각 단어의 감정 분류와 그 정도를 알 수 있어야 함
#   텍스트와 감성지수가 사전에 정의되어 있어야 함

import glob

# ! pip install afinn
from afinn import Afinn

#imdb 데이터셋 5만건의 학습용,검증용 데이터셋 긍정,부정 리뷰로 라벨링되어 있음.
#긍정리뷰데이터 20번째 내용
# glob.glob 특정한 패턴의 파일만 선택하는 함수
pos_review=(glob.glob("c:/workspace3/data/train/pos/*.txt"))[20]

f = open(pos_review)
#파일을 읽음
lines1 = f.readlines()[0]
f.close()

In [2]:
print(lines1)

This Night Listener is better than people are generally saying. It has weaknesses, and it seems to be having a genre identity crisis, no doubt, but I think its creepy atmosphere and intriguing performances make up for this. The whole thing feels like one of those fireside "this happened to a friend of a friend of mine" ghost stories. One big complaint about the movie is the pacing: but the slow and sometimes awkward pacing is deliberate. Everything that unfolds in this movie is kept well within the realm of possibility, and real life just sort of plods along혰no? So there are no flashy endings or earth-shattering revelations, no "showdown" scenes. Thank Heaven. You have to get into the zone when watching this movie, forget your reservations and your expectations of what makes a (conventionally)good movie. Williams isn't terrific, but he easily meets the needs of the story, plus his character is supposed to be somewhat generic ("No One") as he is the Everyman, the avatar by which we ours

In [3]:
#감성분석 객체
afinn = Afinn()

#텍스트 전처리 후 감성점수 산출
afinn.score(lines1)

8.0

In [4]:
files=list(glob.glob('c:/workspace3/data/train/pos/*.txt')[:10])
files

['c:/workspace3/data/train/pos\\0_9.txt',
 'c:/workspace3/data/train/pos\\10000_8.txt',
 'c:/workspace3/data/train/pos\\10001_10.txt',
 'c:/workspace3/data/train/pos\\10002_7.txt',
 'c:/workspace3/data/train/pos\\10003_8.txt',
 'c:/workspace3/data/train/pos\\10004_8.txt',
 'c:/workspace3/data/train/pos\\10005_7.txt',
 'c:/workspace3/data/train/pos\\10006_7.txt',
 'c:/workspace3/data/train/pos\\10007_7.txt',
 'c:/workspace3/data/train/pos\\10008_7.txt']

In [5]:
#학습용 긍정리뷰 10개 파일만 테스트

afinn=Afinn() #감성분석 함수

for i in files:
    f=open(i) #파일 오픈
    lines1=f.readlines()[0] #리스트의 첫번째 문자열
    print(afinn.score(lines1)) #감성점수
    f.close()

-1.0
2.0
19.0
3.0
14.0
8.0
22.0
28.0
13.0
5.0


In [6]:
#부정리뷰데이터 20번째 내용

neg_review=(glob.glob("c:/workspace3/data/train/neg/*.txt"))[20]
f = open(neg_review)
lines2 = f.readlines()[0]
f.close()

print(lines2)

The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil.


In [7]:
afinn.score(lines2)

-4.0

In [8]:
files=list(glob.glob('c:/workspace3/data/train/neg/*.txt')[:10])
files

['c:/workspace3/data/train/neg\\0_3.txt',
 'c:/workspace3/data/train/neg\\10000_4.txt',
 'c:/workspace3/data/train/neg\\10001_4.txt',
 'c:/workspace3/data/train/neg\\10002_1.txt',
 'c:/workspace3/data/train/neg\\10003_1.txt',
 'c:/workspace3/data/train/neg\\10004_3.txt',
 'c:/workspace3/data/train/neg\\10005_3.txt',
 'c:/workspace3/data/train/neg\\10006_4.txt',
 'c:/workspace3/data/train/neg\\10007_1.txt',
 'c:/workspace3/data/train/neg\\10008_2.txt']

In [9]:
#학습용 부정리뷰 10개 파일만 테스트

afinn=Afinn() #감성분석 함수
for i in files:
    f=open(i) #파일 오픈
    lines1=f.readlines()[0] #리스트의 첫번째 문자열
    print(afinn.score(lines1)) #감성점수
    f.close()

6.0
-4.0
9.0
5.0
-7.0
1.0
13.0
4.0
7.0
6.0


In [10]:
# 2.기계학습으로 감성분석(시간이 매우 오래 걸림)

import glob

#긍정 텍스트 로딩
pos_review=(glob.glob("c:/workspace3/data/train/pos/*.txt")[:100])

lines_pos=[]

for i in pos_review:
    try:
        f = open(i)
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except :
        continue

len(lines_pos)

100

In [11]:
#부정 텍스트 로딩

neg_review=(glob.glob("c:/workspace3/data/train/neg/*.txt")[:100])

lines_neg=[]

for i in neg_review:
    try:
        f = open(i)
        temp = f.readlines()[0]
        lines_neg.append(temp)
        f.close()
    except :
        continue

len(lines_neg)

100

In [12]:
#긍정,부정 리뷰를 합침

total_text=lines_pos+lines_neg

len(total_text)

200

In [13]:
import numpy as np
from nltk.corpus import stopwords

#긍정,부정 클래스 라벨링
x = np.array(["pos", "neg"])
class_Index=np.repeat(x, [len(lines_pos), len(lines_neg)], axis=0)
stop_words = stopwords.words('english')

In [14]:
#단어들에 Tfidf 가중치를 부여한 후 문서-단어 매트릭스로 바꿈

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words=stop_words).fit(total_text)

X_train_vectorized = vect.transform(total_text)
X_train_vectorized.index = class_Index

In [15]:
import pandas as pd

#데이터프레임으로 변환

df=pd.DataFrame(X_train_vectorized.toarray(), columns=vect.vocabulary_.keys())
df.head()

,bromwell,high,cartoon,comedy,ran,time,programs,school,life,teachers,...,zombified,auteur,ample,opportunities,golden,geist,uttered,downloading,midget,tricking
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
for idx,value in enumerate(X_train_vectorized[0].toarray()[0]):  
    if value>0:
        print(idx, value)

64 0.10070285188783222
189 0.10070285188783222
212 0.09513664322740871
585 0.06603504311540802
773 0.43419192948676516
816 0.10070285188783222
893 0.09081915636680153
1038 0.07740781722251895
1066 0.09513664322740871
1105 0.0708975934136058
1921 0.09081915636680153
2034 0.0708975934136058
2128 0.06498892497285297
2174 0.10854798237169129
2210 0.10070285188783222
2727 0.3478134336932994
2903 0.10070285188783222
3008 0.10854798237169129
3013 0.10854798237169129
3249 0.07944638225969057
3320 0.09081915636680153
3387 0.049185844603996734
3407 0.06831396493884356
3563 0.04746484661907397
3828 0.040035828354977356
4031 0.031912297780574375
4179 0.09081915636680153
4242 0.10854798237169129
4288 0.10070285188783222
4358 0.10854798237169129
4480 0.10070285188783222
4487 0.10854798237169129
4618 0.10070285188783222
4664 0.09513664322740871
4676 0.10854798237169129
4757 0.09081915636680153
4772 0.10070285188783222
4872 0.06305246292974673
4956 0.10854798237169129
4992 0.10854798237169129
5001 0.0

In [17]:
#로지스틱 회귀 모형

from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(random_state=10)

logit.fit(X_train_vectorized, class_Index)

LogisticRegression(random_state=10)

In [18]:
#긍정 리뷰들을 하나씩 불러와서 실험

def pos_review(model):

    count_all=0
    count=0
    num=100

    tests1=[]

    for idx in range(0,num):
        pos_review_test=(glob.glob("c:/workspace3/data/test/pos/*.txt"))[idx]
        f = open(pos_review_test, 'r',encoding="utf-8")
        tests1.append(f.readlines())
        f.close()

    for test in tests1:
        pred = model.predict(vect.transform(test))
        result=pred[0]
        if result=="pos":
            count+=1
        count_all += 1
    rate= count*100/count_all

    print(f"긍정 분류정확도:{rate:.1f}%")

In [19]:
#부정 리뷰들을 하나씩 불러와서 실험

def neg_review(model):

    count_all=0
    count=0
    num=100

    tests2=[]

    for idx in range(0,num):
        neg_review_test=(glob.glob("c:/workspace3/data/test/neg/*.txt"))[idx]
        f = open(neg_review_test, 'r',encoding="utf-8")
        tests2.append(f.readlines())
        f.close()

    for test in tests2:
        preds = model.predict(vect.transform(test))
        result=preds[0]
        if result=="neg":
            count+=1
        count_all+=1
    rate= count*100/count_all

    print("부정 분류정확도:{0:.1f}%".format(rate))

pos_review(logit)
neg_review(logit)

긍정 분류정확도:66.0%
부정 분류정확도:81.0%


In [20]:
#의사결정나무 모형

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=10)

tree.fit(X_train_vectorized, class_Index)

pos_review(tree)
neg_review(tree)

긍정 분류정확도:39.0%
부정 분류정확도:66.0%


In [21]:
#랜덤포레스트

from sklearn.ensemble import RandomForestClassifier

#10개의 트리로 구성된 랜덤 포레스트

forest = RandomForestClassifier(n_estimators=10, random_state=10)

forest.fit(X_train_vectorized, class_Index)

pos_review(forest)
neg_review(forest)

긍정 분류정확도:44.0%
부정 분류정확도:75.0%


In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=2)

knn.fit(X_train_vectorized, class_Index)

pos_review(knn)
neg_review(knn)

긍정 분류정확도:34.0%
부정 분류정확도:85.0%


In [23]:
#인공신경망

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=10)

mlp.fit(X_train_vectorized, class_Index)

pos_review(mlp)
neg_review(mlp)

긍정 분류정확도:58.0%
부정 분류정확도:79.0%


In [24]:
#SVM 모형

from sklearn.svm import SVC

svm = SVC(random_state=10)

svm.fit(X_train_vectorized, class_Index)

pos_review(svm)
neg_review(svm)

긍정 분류정확도:63.0%
부정 분류정확도:87.0%


In [25]:
#! pip install textblob

In [26]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

#train = [ ('샌드위치 좋아해요', 'pos'), ('너무 좋은 곳이예요', 'pos'), ('너무 맛있어요', 'pos'), ('내가 제일 좋아하는 곳이예요', 'pos'), ('친한 친구예요', 'pos'), ('이 사람은 믿을 수 없어요', 'neg'), ('별로 안좋은 곳이네요', 'neg'), ("맛이 별로네요", 'neg'), ('경치가 별로예요', 'neg'), ('별로 볼게 없네요', 'neg') ]

#test = [ ('최고의 음료수', 'pos'), ('별로 안좋아요', 'neg'), ('이번주는 컨디션이 안좋아요', 'neg'), ('너무 좋아요', 'pos'), ('나와 가장 친해요', 'pos'), ("믿을 수 없어요", 'neg') ]

train = [
    ('I love this sandwich.', 'pos'),
    ('This is an amazing place!', 'pos'),
    ('I feel very good about these beers.', 'pos'),
    ('This is my best work.', 'pos'),
    ('What an awesome view', 'pos'),
    ('I do not like this restaurant', 'neg'),
    ('I am tired of this stuff.', 'neg'),
    ("I can't deal with this", 'neg'),
    ('He is my sworn enemy!', 'neg'),
    ('My boss is horrible.', 'neg')
]

test = [
    ('The beer was good.', 'pos'),
    ('I do not enjoy my job', 'neg'),
    ('I am not feeling dandy today.', 'neg'),
    ('I feel amazing!', 'pos'),
    ('Gary is a friend of mine.', 'pos'),
    ("I can't believe I'm doing this.", 'neg')
]

cl = NaiveBayesClassifier(train)

#print(cl.classify('맛있는 음식이네요'))
#print(cl.classify("피자맛이 별로네요"))

print(cl.classify('Their burgers are amazing'))  
print(cl.classify("I don't like their pizza."))  

pos
neg


In [27]:
#여러 문장을 종합하여 부정으로 분류
#blob = TextBlob("맛있는 음식이네요. 피자는 별로네요. 서비스는 좋네요.", classifier=cl)

blob = TextBlob("The beer was amazing. But the hangover was horrible. My boss was not happy.", classifier=cl)

blob.classify()  # "neg"  

'neg'

In [28]:
#개별 문장으로 분류

for sentence in blob.sentences:
    print(sentence, '==>', sentence.classify())

# "pos", "neg", "neg"

for row in test:
    print(row[0],'==>', cl.classify(row[0]))

cl.accuracy(test)

The beer was amazing. ==> pos
But the hangover was horrible. ==> neg
My boss was not happy. ==> neg
The beer was good. ==> pos
I do not enjoy my job ==> neg
I am not feeling dandy today. ==> neg
I feel amazing! ==> pos
Gary is a friend of mine. ==> neg
I can't believe I'm doing this. ==> neg


0.8333333333333334

In [29]:
cl.show_informative_features(5)

Most Informative Features
          contains(this) = True              neg : pos    =      2.3 : 1.0
          contains(this) = False             pos : neg    =      1.8 : 1.0
          contains(This) = False             neg : pos    =      1.6 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0


In [30]:
# Most Informative Features
#           contains(this) = True              neg : pos    =      2.3 : 1.0
#           this가 포함된 경우 부정:긍정=2.3:1.0
#           contains(this) = False             pos : neg    =      1.8 : 1.0
#           this가 포함되지 않은 경우 긍정:부정=1.8:1.0
#           contains(an) = False             neg : pos    =      1.6 : 1.0
#           contains(This) = False             neg : pos    =      1.6 : 1.0
#           contains(I) = True              neg : pos    =      1.4 : 1.0